<a href="https://colab.research.google.com/github/ldennis1/BSAN6070--CA02/blob/main/CA02_LaurenDennis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objective: Naïve Bayes Classifier to create a spam filtering email algorithm 

In [2]:
#importing necessary packages 
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [3]:
'''This function builds a Dictionary of most common 3000 words from all the email content. 
First it adds all words and symbols in the dictionary. 
Then it removes all non-alpha-numeric characters and any single character alpha-numeric characters. 
After this is complete it shrinks the Dictionary by keeping only most common 3000 words in the dictionary. 
It returns the Dictionary.'''

def make_Dictionary(init_dir):   
  total_words = [] #step to create dict of all words from email content 
  emails = [os.path.join(init_dir,f) for f in os.listdir(init_dir)] 
  for allmail in emails:
    with open(allmail) as m:
      for line in m:
        words = line.split() #splitting sentences into lists 
        total_words += words
  dictionary = Counter(total_words) #gives us the frequency of the words in the dictionary
  list_to_remove = list(dictionary)

  for charac in list_to_remove:
    if charac.isalpha() == False: #removes non-alpha-numeric
      del dictionary[charac]
    elif len(charac) == 1: #removes single character alpha-numeric characters 
      del dictionary[charac]
  dictionary = dictionary.most_common(3000) #most frequent 3000 words to the dictionary
  return dictionary

In [8]:
"""This function extracts feature columns and populates their values 
(Feature Matrix of 3000 comumns and rows equal to the number of email files). 
The function also analyzes the File Names of each email file and decides if it's 
a Spam or not based on the naming convention. Based on this the function also 
creates the Labelled Data Column. This function is used to extract the training dataset 
as well as the testing dataset and returns the Feature Dataset and the Label column."""

def extract_features(mail_dir): 
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)] 
  features_matrix = np.zeros((len(files),3000)) #3000 columns and rows to correspond with initial directory of 3000
                                                #most common words
  train_labels = np.zeros(len(files)) 
  count = 1;
  docID = 0;
  for fil in files: #uses a for loop and enumerate to number the items in the dict 
    with open(fil) as fi:
      for i, line in enumerate(fi):
        if i ==2:
          words = line.split()
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word)
      train_labels[docID] = 0;
      filepathTokens = fil.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"): #iterates through file names to decide if spam/real based on the given naming legend
        train_labels[docID] = 1;
        count = count + 1
      docID = docID + 1
  return features_matrix, train_labels

In [5]:
"""The section is the main Program that calls the above two functions and gets executed first. 
First it "trains" the model using model.fit function and Training Dataset. 
After that it scores the Test Data set by running the Trained Model with the Test Data set. 
At the end it prints the model performance in terms of accuracy score."""

train_dir = '/content/drive/MyDrive/6070 BRAHMA/CA02/Data/train-mails' #mounted from google drive 
test_dir = '/content/drive/MyDrive/6070 BRAHMA/CA02/Data/test-mails' #mounted from google drive 

dictionary = make_Dictionary(train_dir)

print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(train_dir) #trains the model
test_features_matrix, test_labels = extract_features(test_dir) #tests the trained model with the test data

model = GaussianNB() #implements the Gaussian Naive Bayes classifier 

print ("Training Model using Gaussian Naibe Bayes algorithm .....")
model.fit(features_matrix, labels) #passes the functions into the Gaussian Naive Bayes Classifier model 
print ("Training completed")
print ("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels)) #shows the calculation of predictions the model got correct

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9615384615384616
